<a href="https://colab.research.google.com/github/cbachen1997/CAUstudy/blob/Keras_TripletLoss/Keras_hard_tripletloss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)

In [2]:
#更改运行目录
import os
os.chdir("/content/gdrive/My Drive/AndrewNgTest/Keras_tripletloss")
print (os.path.abspath('.')) #获取当前工作目录路径
print(os.listdir('.'))

/content/gdrive/My Drive/AndrewNgTest/Keras_tripletloss
['inception_blocks_v2.py', 'train_face.h5', 'test_happy.h5', 'fr_utils.py', 'train_happy.h5', '__pycache__', 'FRmodel.png', 'images', 'weights', 'Triplet_loss_KERAS_semi_hard_from_TF.ipynb', 'Triplet_loss_KERAS_semi_hard_from_TF (1).ipynb', 'Keras_tripletloss.ipynb', 'Keras_tripletloss (1).ipynb']


In [131]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
import h5py

#模型绘制
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#数据格式(m,c,w,h)
K.set_image_data_format('channels_first')

import time
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import fr_utils
from inception_blocks_v2 import *#在这 o
import random

import matplotlib.pyplot as plt

%matplotlib inline 
#plot后直接画图，不用再show
%load_ext autoreload
#自动装入所有模块
%autoreload 2
#每运行一个cell都会重新import所有module
np.set_printoptions(threshold=1000000)#np.nan输出所有的值，不要省略号，报错未知

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:
#调用inception_blocks_v2
FRmodel = faceRecoModel(input_shape=(3,96,96))
# FRmodel=testModel(input_shape=(3,96,96))
#输入(m,3,96,96)
#输出(m,128)向量
#打印模型的总参数数量
# print("参数数量：" + str(FRmodel.count_params()))

In [5]:
#绘制模型
# %matplotlib inline
# plot_model(FRmodel, to_file='FRmodel.png',show_shapes=True)

In [6]:
#人脸数据集的输入
def load_face():
  train_dataset = h5py.File('train_face.h5', "r")
  train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
  train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
  train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))

  return train_set_x_orig, train_set_y_orig

In [7]:
train_set_x_orig, train_set_y_orig=load_face()

In [9]:
# train_set_y_orig[0][1].decode('utf-8')
namecount = {'sebastiano': 68, 'felix': 66, 'benoit': 51, 'arnaud': 31, 'bertrand': 61, 'kian': 39, 'younes': 52, 'dan': 58}
# nameindex = {'sebastiano': 0, 'felix': 1, 'benoit': 2, 'arnaud': 3, 'bertrand': 4, 'kian': 5, 'younes': 6, 'dan': 7}

In [10]:
#建立索引函数
def create_index(train_set_y):
#=============获取数据集中人的集合================
  name=np.unique(train_set_y)
  #解码获得姓名列表
  namelist=[]
  for i in name:
    namelist.append(i.decode('utf8'))
#==============获取每个人的数据索引=========================
  #初始化
  person_index=locals()
  for i,name in enumerate(namelist):
    person_index[name]=[]
  for j in range(train_set_y.shape[1]):
    name=train_set_y[0][j].decode('utf-8')
    person_index[name].append(j)
  return namelist,person_index

In [11]:
#随机抽取batch
def sample_people(namelist,person_index,num_person,num_picture):
  #随机选定指定数量的人数
  total=num_person*num_picture
  person=random.sample(namelist,num_person)
  #从人的列表中随机抽5个
  index=locals()
  for i in person:
    temp=[]
    index[i]=random.sample(list(person_index[i]),num_picture)
  return person,index,total

In [12]:
#计算batch中各元素的embedding编码
def calculate_embedding(train_set_x,model,namelist,person_index,numperson,numpicture):
  person,index,total=sample_people(namelist=namelist,person_index=person_index,num_person=numperson,num_picture=numpicture)
  embed_array=np.zeros((total,128))
  embed_index=np.zeros((1,total))
  for num,name in enumerate(person):
    #获取每个人的数字
    temp=index[name]
    for num_index,j in enumerate(temp):
      #获取每个人对应的影像
      # print(j)
      img=train_set_x[j]
      img=np.around(np.transpose(img,(2,0,1))/255.0,decimals=12)
      x_train=np.array([img])#shape=(1,3,96,96)
      #投影到embedding空间
      embedding=model.predict_on_batch(x_train)
      embed_array[5*num+num_index,:]=embedding
      embed_index[0,5*num+num_index]=j
  return embed_array,embed_index
  #获得一个(5,5,128)的编码矩阵，(x,y,vector)，x代表人，y代表人对应的抽出来的图片，vector为图片经过embedding得到的结果
  #embed_index为编码对应的图像索引

In [13]:
#计算距离矩阵
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
def D_mat(X):
  """
  X是batch中所有的embedding
  """
  #距离矩阵计算方法https://zhuanlan.zhihu.com/p/77034615
  # print(X.shape)
  # A是一个向量矩阵：euclidean代表欧式距离
  distA=pdist(X,metric='euclidean') 
  distB = squareform(distA)
  return distB
#获取最大距离
def get_hard_positive(Dmat,embed_index,anchor,num_per_person):
  range_=int(anchor/num_per_person)
  remainder=anchor%num_per_person
  temp=Dmat
  #positive矩阵
  p_mat=Dmat[range_*5:range_*5+5,range_*5:range_*5+5]
  #negative矩阵
  n_mat=np.delete(Dmat[range_*5:range_*5+5,:],[i for i in range(range_*5,range_*5+5)],1)
#===========================获取hard_positive===================================
  # print(p_mat)
  hp=np.argmax(p_mat[remainder][:])#p_mat中hard_positive的位置
  # print(p_mat[remainder][:])
  # print(hp)
  hp_index=5*range_+hp
  # print(hp_index)
  hp_index=int(embed_index[:,hp_index])
#===========================获取hard_negative===================================
  np_=np.argmin(n_mat[remainder][:])
  # print(n_mat[remainder][:])
  if (np_+5)>5*range_+4:
    np_index=np_+5
  else:
    np_index=np_
  # print(np_index)
  np_index=int(embed_index[:,np_index])
#============================获得一组困难三元组
  anchor_index=int(embed_index[:,anchor])
  return [anchor_index,hp_index,np_index]

In [14]:
#从batch中构建困难三元组
def get_hard_triplet(embed_array,embed_index):
  #获取距离矩阵
  distance=D_mat(embed_array)
  #三元组编号数组
  hard_triplet=[]
  #构建PK个三元组，P为人数，K为每人选取照片个数
  for i in range(embed_array.shape[0]):
    hard_triplet.append(get_hard_positive(distance,embed_index,anchor=i,num_per_person=5))
  return hard_triplet

In [105]:
def triplet_loss(y_pred,alpha):
  """ 
  参数：
      y_true -- true标签，当你在Keras里定义了一个损失函数的时候需要它，但是这里不需要。
      y_pred -- 列表类型，包含了如下参数：
          anchor -- 给定的“anchor”图像的编码，维度为(None,128)
          positive -- “positive”图像的编码，维度为(None,128)
          negative -- “negative”图像的编码，维度为(None,128)
      alpha -- 超参数，阈值
  
  返回：
      loss -- 实数，损失的值
  """
  # print(y_pred)
  #获取anchor, positive, negative的图像编码
  anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
  #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
  pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1,keepdims=True)#二范数，-1代表倒数第一个维度即128
  #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
  neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1,keepdims=True)
  #第三步：减去之前的两个距离，然后加上alpha
  basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),tf.constant(alpha))
  #通过取带零的最大值和对训练样本的求和来计算整个公式
  loss = tf.maximum(basic_loss,0)
  # loss = basic_loss
  # print(loss)
  return loss


In [106]:
#构建困难三元组流程
def process_triplet(train_set_x,train_set_y,model,num_person_,num_picture_):
  #建立索引和名字列表
  namelist,person_index=create_index(train_set_y)
  #随机抽取batch
  person,index,total=sample_people(namelist,person_index=person_index,num_person=num_person_,num_picture=num_picture_)#修改此处来修改batchsize
  #计算batch中所有图片的编码
  embed_array,embed_index=calculate_embedding(train_set_x,model,namelist,person_index,numperson=num_person_,numpicture=num_picture_)
  #获得困难三元组
  hard_triplet=get_hard_triplet(embed_array,embed_index)
  return hard_triplet,total

In [107]:
#获取三元组数据,及标签
def get_triplet_data(triplet):
  y=np.zeros((3,3,96,96))
  labels=[]
  flag=0
  for i in triplet:
    # print(np.transpose(train_set_x_orig[i],[2,0,1]).shape
    y[flag,:]=np.transpose(train_set_x_orig[i],[2,0,1])
    flag+=1
  return y

In [152]:
def create_train_model(FRmodel,alpha):
  y_pred=[]
  anchor_input = Input((3,96,96)) 
  positive_input = Input((3,96,96))
  negative_input = Input((3,96,96))
  # FRmodel = faceRecoModel(input_shape=(3,96,96))
  # an = Model(anchor_input, FRmodel(anchor_input))
  anchor_out = Model(anchor_input, FRmodel(anchor_input))
  positive_out = Model(positive_input, FRmodel(positive_input))
  negative_out = Model(negative_input, FRmodel(negative_input))
  # anchor_out = FRmodel(anchor_input)
  # positive_out = FRmodel(positive_input)
  # negative_out = FRmodel(negative_input)
  y_pred.append(anchor_out.output)
  y_pred.append(positive_out.output)
  y_pred.append(negative_out.output)

  loss = Lambda(lambda x: triplet_loss(x, alpha), output_shape=[1])(y_pred)
  # y_true=np.zeros((total,3)).tolist()
  # alpha=0.2
  return Model(inputs = [anchor_input,positive_input,negative_input], outputs=loss)

In [120]:
#训练模型
def train_model(FRmodel,alpha=0.2):
  #开始时间
  tic = time.clock()
  num_times=10
  model=create_train_model(FRmodel,alpha)
  # model.add_loss(model.output)
  model.compile(optimizer = 'adam', loss=lambda t,p:p, metrics = 'accuracy')
  # model.add_loss(triplet_loss())
  '''
  func(y_t, y_pred):
    y_t 随机 无用 
    loss = y_pred
    return loss
  '''
  model.summary()
  for i in range(num_times):
    #获得一个batch的困难三元组
    print('正在训练第'+str(i)+'个batch')
    hard_tri,total=process_triplet(train_set_x_orig,train_set_y_orig,FRmodel,num_person_=8,num_picture_=10)
    # y_pred=[]
    y_label=np.zeros((total,1))#y用不着
    # y_label=np.array(y_label)
    flags=0
    x_input = [ np.zeros((total, 3, 96, 96)), np.zeros((total, 3, 96, 96)), np.zeros((total, 3, 96, 96)) ]
    #构建输入的x
    for i in hard_tri:
      triplet = i
      triplet = list(map(int, triplet))
      y=get_triplet_data(triplet)#(3,3,96,96)
      # y=np.array(y)
      for idx in range(3):
        x_input[idx][flags] = y[idx]
      # y_pred.append(y)
      flags+=1
    #y_pred为x，y_label为y
    
    model.fit(x=x_input,y=y_label,batch_size=total,epochs=5)

In [159]:
#开始时间
tic = time.clock()

#加载权值
# fr_utils.load_weights_from_FaceNet(FRmodel)
#模型训练
# FRmodel.compile(optimizer='adam')
train_model(FRmodel,alpha=0.3) 
#结束时间
toc = time.clock()

#计算时差
minium = toc-tic

print("执行了：" + str(int(minium / 60)) + "分" + str(int(minium%60)) + "秒")

Model: "functional_251"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_152 (InputLayer)          [(None, 3, 96, 96)]  0                                            
__________________________________________________________________________________________________
input_153 (InputLayer)          [(None, 3, 96, 96)]  0                                            
__________________________________________________________________________________________________
input_154 (InputLayer)          [(None, 3, 96, 96)]  0                                            
__________________________________________________________________________________________________
FaceRecoModel (Functional)      (None, 128)          3743280     input_152[0][0]                  
                                                                 input_153[0][0]     

In [123]:
FRmodel.save('my_FRmodel.h5')
#储存模型

In [161]:
database = {}
database["danielle"] = fr_utils.img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = fr_utils.img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = fr_utils.img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = fr_utils.img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = fr_utils.img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = fr_utils.img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = fr_utils.img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = fr_utils.img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = fr_utils.img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = fr_utils.img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = fr_utils.img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = fr_utils.img_to_encoding("images/arnaud.jpg", FRmodel)

In [ ]:
#载入模型
from keras.models import  load_model
testmodel=load_model('my_FRmodel.h5')


In [128]:
def verify(image_path, identity, database, model):
    """
    对“identity”与“image_path”的编码进行验证。
    
    参数：
        image_path -- 摄像头的图片。
        identity -- 字符类型，想要验证的人的名字。
        database -- 字典类型，包含了成员的名字信息与对应的编码。
        model -- 在Keras的模型的实例。
        
    返回：
        dist -- 摄像头的图片与数据库中的图片的编码的差距。
        is_open_door -- boolean,是否该开门。
    """
    #第一步：计算图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = fr_utils.img_to_encoding(image_path, model)
    
    #第二步：计算与数据库中保存的编码的差距
    dist = np.linalg.norm(encoding - database[identity])
    
    #第三步：判断是否打开门
    if dist < 0.7:
        print("欢迎 " + str(identity) + "回家！")
        is_door_open = True
    else:
        print("经验证，您与" + str(identity) + "不符！")
        is_door_open = False
    
    return dist, is_door_open

In [ ]:
path="images/camera_2.jpg"
verify(path,"benoit",database,FRmodel)
img=plt.imread(path)
plt.imshow(img)

In [156]:
def who_is_it(image_path, database,model):
    """
    根据指定的图片来进行人脸识别
    
    参数：
        images_path -- 图像地址
        database -- 包含了名字与编码的字典
        model -- 在Keras中的模型的实例。
        
    返回：
        min_dist -- 在数据库中与指定图像最相近的编码。
        identity -- 字符串类型，与min_dist编码相对应的名字。
    """
    #步骤1：计算指定图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = fr_utils.img_to_encoding(image_path, model)
    
    #步骤2 ：找到最相近的编码
    ## 初始化min_dist变量为足够大的数字，这里设置为100
    min_dist = 100
    
    ## 遍历数据库找到最相近的编码
    for (name,db_enc) in database.items():
        ### 计算目标编码与当前数据库编码之间的L2差距。
        dist = np.linalg.norm(encoding - db_enc)
        
        ### 如果差距小于min_dist，那么就更新名字与编码到identity与min_dist中。
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    # 判断是否在数据库中
    if min_dist > 0.7:
        print("抱歉，您的信息不在数据库中。")
        
    else:
        print("姓名" + str(identity) + "  差距：" + str(min_dist))
    
    return min_dist, identity

In [ ]:
for i in range(5):
  path = "images/camera_"+str(i) + ".jpg"
  who_is_it(path, database, FRmodel)
  img=plt.imread(path)
  plt.imshow(img)